# Zillow Data Visualizer
## *Edward Krueger*

This notebook uses Quandl's API client to pull data from the Zillow Real Estate Research from Quandl.
It loads that data into a ```pandas``` DataFrame and visualizes it using the ```bokeh``` package. 

There are many locations available. For a complete list, or to search for a location, I have build the documentation in ```build_zillow_docs.ipynb```. There are also many types of data available. I will focus on Median Rental Price and Median Listing Price, but my visualizer will work with other codes.

Here is a list of indicator code prefixes and their descriptions.
These correspond to types of data.
* AOI - Age of Inventory
* BSI - Buyer Seller Index
* SALES - Volume of Sale
* SASALES - Volume of Sales Seasonally Adj.
* LPC - Listings Price Cuts
* MHI - Market Health Index 
* MLP - Median Listing Price
* MLPF - Median Listing Price Per Square Foot
* MPC - Median Price Cut
* MPPR - Median Price Of Reduction
* MRP - Median Rental Price
* MRPF - Median Rental Price Per Square Foot
* MSP - Median Sold Price
* MSPF - Median Sold Price Per Square Foot
* MVALF - Median Value Per Square Foot
* PHDV - Percent Of Homes Decreasing In Values
* PDIV - Percent Of Homes Increasing In Values
* PHS - Percent Of Homes Selling For Gain/Loss
* PLPR - Percent Of Listings With Price Reductions
* PTPFH - Percent Transactions That Are Previously Foreclosured Homes
* PRR - Price To Rent Ratio
* TURN - Turnover
* USR - Unsold Renos
* ZHVI - Zillow Home Value Index
* ZRI - Zillow Rental Index

## Setup

### Import and configure packages

In [1]:
# import packages to unzip the list of Quandl Codes
from io import BytesIO
from zipfile import ZipFile
import urllib.request

# import and configure pandas
import pandas as pd
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 1000)

# import quandl and configure API key
import quandl
quandl.ApiConfig.api_key = "qTtZzvXVi7jgbHx5tng8"

# import functions to visualize the data and set Bokeh to output to the notebook
from bokeh.plotting import figure, output_file, show
from bokeh.models import NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Create functions to load and visualize the data

In [2]:
# takes the url (as a string) to the location of the zipped Quandl Codes .csv, uzips and loads it as a pd DataFrame
def get_quandl_codes(url_str = "https://www.quandl.com/api/v3/databases/ZILLOW/codes?api_key=qTtZzvXVi7jgbHx5tng8"):
    
    # open the zip file as a ZipFile
    url = urllib.request.urlopen(url_str)
    zip_file = ZipFile(BytesIO(url.read()))
    
    # find the name of the only file and open it
    names = zip_file.namelist()
    name = names[0]
    
    # load into pandas
    df = pd.read_csv(zip_file.open(name), header = None, low_memory = False)
    
    # rename to columns
    df.columns = ["code", "description"]
    
    # strip the first par of the description
    df["description"] = df["description"].apply(lambda x : x.split(":")[1])
    
    # 
    df["zillow_code"] = df["code"].apply(lambda x : x.split("/")[1])
    df["area_cat_code"] = df["zillow_code"].apply(lambda x : x.split("_")[0])
    df["indicator_code"] = df["zillow_code"].apply(lambda x : x.split("_")[1])
    
    return df

# Gets a dictionary of all timeseries given a area_cat_code, a ind_prefix and a DataFrame of quandl codes.
# area_cat_code is a combination of the area category and the area code: {AREA_CATEGORY}{AREA_CODE} (ex. Z77079)
# ind_prefix is the prefix of the {INDICATOR_CODE} of the series: (ex. "MLP")
# df  is a DataFrame of quandl codes generated by load_quandl_codes()
def filter_by_area_prefix(area_cat_code, ind_prefix, df):
    
    # subset
    df = df[df["area_cat_code"] == area_cat_code]
    df = df[df["code"].str.contains(ind_prefix) & ~ df["code"].str.contains(ind_prefix + "F")]
    
    # create columns
    df["series_type"] = df["description"].apply(lambda x : x.split("-")[0].strip())
    df["property_type"] = df["description"].apply(lambda x : x.split("-")[1].strip())
    df["area"] = df["description"].apply(lambda x : x.split("-")[2].strip())

    # convert to dictionary
    #dct = pd.Series(df["description"].values, index = df["code"]).to_dict()
    
    df = df.set_index("code")
    
    return df

# Takes a list (or dict_keys) of Quandl Codes and returns a pd DataFrame containing all of the data
def get_data(quandl_codes):
    
    data_keys = list(quandl_codes)
    
    out_df = pd.DataFrame()
    
    for code in quandl_codes:
        out_df[code] = quandl.get(code)["Value"]

    return out_df

# creates the plot
def create_plot(s_codes):
    
    # filter the codes
    s_codes_info = s_codes
    s_codes_list = list(s_codes_info.index)
    
    # get the data
    df = get_data(s_codes_list)

    # create a color iterator
    # this palette only has 10 colors, but we won't want to visualize more series than that
    from bokeh.palettes import Category10 as palette
    palette = palette[10]
    colors = iter(palette)

    # find the title
    idx_0 = df.columns[0]
    title = s_codes_info.loc[idx_0]["series_type"] + " - " + s_codes_info.loc[idx_0]["area"]

    # get the scaling for the y axis
    y_max = df.dropna(how = "any").values.max() * 1.5
    y_min = df.dropna(how = "any").values.min() * .7

    #create the plot
    p = figure(width = 800, height = 500,
               tools = "pan,wheel_zoom,box_zoom,reset", title =  title,
               x_axis_label = "Date", x_axis_type = "datetime",
               y_axis_label = "Value", y_range =(y_min, y_max))

    # format the y-axis
    p.yaxis[0].formatter = NumeralTickFormatter(format= "$0,0")

    # add the line
    for code in s_codes_list:
        p.line(df.index, df[code], legend = s_codes_info.loc[code]["property_type"], line_color = next(colors))

    # move the ledgend
    p.legend.location = "top_left"

    return p

## Load and Visualize the Data

In [3]:
# load all of the codes
codes = get_quandl_codes()

I'm interested in seeing how hurricane Harvey of August and September 2017 effected house prices in the flood-prone Meyerland neighborhood. I found that the code is "N11814" using ```build_zillow_docs.ipynb```.

### Visualize Median Listing Price for Meyerland, Houston TX

In [4]:
# get and visualize all MLP data for N11814
s_codes_info = filter_by_area_prefix(area_cat_code = "N11814", ind_prefix = "MLP", df = codes)
s_codes_list = list(s_codes_info.index)
p = create_plot(s_codes_info)
show(p)

As expected, there was a drop around the time of the flood. Prices continued to drop afterwards and may be stabilizing as of late 2017 / early 2018.

### Visualize Median Renal Price for Austin Metro

We often hear that rental prices have increased in Austin in recent years. Lets first have a look at at the Austin metro area. I found the code using ```build_zillow_docs.ipynb``` to be "M31".

In [5]:
# get and visualize all MRP data for N11814
s_codes_info = filter_by_area_prefix(area_cat_code = "M31", ind_prefix = "MRP", df = codes)
s_codes_list = list(s_codes_info.index)
p = create_plot(s_codes_info)
show(p)

Now lets have a look at the City of Austin. I found the code is "C11734".

In [6]:
# get and visualize all MRP data for City of Austin City
s_codes_info = filter_by_area_prefix(area_cat_code = "C11734", ind_prefix = "MRP", df = codes)
s_codes_list = list(s_codes_info.index)
p = create_plot(s_codes_info)
show(p)

As expected the prices have risen. Somewhat surprisingly, the city and metro area price seem to be rising at about the same rate. This seems to indicate prices are raising in the suburbs too. Also of note is the rapid growth of studio prices in Austin. However, this is probably because Zillow categorizes even luxury lofts as studios if they don't have a separate bedroom.